## ReMKiT1D input generator - staggered grids with a SOL-KiT style problem

This example uses the fluid model SOL-KiT-like generator to set up fluid runs. See Appendix B of the ReMKiT1D code paper.

The following are dependencies for this example

In [29]:
import numpy as np
import xarray as xr
import holoviews as hv
import matplotlib.pyplot as plt
from holoviews import opts
import panel as pn
import pickle
from sk_comp_thesis import generatorSKThesis

import sys
sys.path.append('../')
import RMK_support.IO_support as io
import RMK_support.dashboard_support as ds


### Grid parameters for Mijin thesis comparison

In [30]:
dx0=0.13542325129584085E+0*8.5/2.5*10.18/9.881556569543156
dxN=0.13542325129584085E+0*0.5/2.5*10.18/9.881556569543156

In [31]:
heatingPower=3.5464790894703255

### Initialize fluid run wrapper using external script

See script for input parameter docs.

In [32]:
rk = generatorSKThesis(dx0 = dx0/4,
                        dxN = dxN/4,
                        Nx = 256,
                        Nh=17,
                        lmax=1,
                        mpiProcs=16,
                        initialTimestep=2.0,
                        nu=0.8/1.09345676,
                        heatingPower=heatingPower,
                        # includedJanevTransitions=["ex","deex","ion","recomb3b"],
                        # includeSpontEmission=True,
                        numNeutrals=1,
                        # loglinExtrap=True,
                        # amjuelCXRate=True,
                        amjuelRates=True,
                        # fixedQRatio=qRatio,
                        # useKineticCorrections=True,
                        # hdf5InputFile="../RMK_SK_comp_staggered_kin/ReMKiT1DVarOutput_99"
                        )


/home/stefan/SMijin/ReMKiT1D-Python-Support/examples/../RMK_support/crm_support.py:110: UserWarning: getTransitionIndicesAndEnergies was unable to find some transition energy indices
  warnings.warn("getTransitionIndicesAndEnergies was unable to find some transition energy indices")


In [33]:
rk.setPETScOptions(cliOpts="-pc_type bjacobi -sub_pc_factor_shift_type nonzero -sub_pc_factor_levels 1",kspSolverType="gmres")

In [34]:
rk.setTimeTargetTimestepping(80000)

In [35]:
rk.writeConfigFile()

### Data analysis

In [ ]:
numFiles =1

In [ ]:
loadFilenames = [rk.hdf5Filepath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]

In [ ]:
loadedData = io.loadFromHDF5(rk.varCont,filepaths=loadFilenames)
loadedData

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=100,dpi=150)

#### Explore data using basic dashboard

In [ ]:
pn.extension(comms="vscode") # change comms if not using VSCode
dashboard = ds.ReMKiT1DDashboard(loadedData,rk.grid)

dashboard.fluid2Comparison().show()
